In [20]:
### 2024 SEP 6 ### script4_seq ## matching

In [21]:
##
# conda install -c conda-forge r-optmatch
##

In [22]:
install.packages("s3")
install.packages("doParallel")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [23]:
library("terra")
library("dplyr")
library("sf")
#install.packages("s3")
library("s3")

library("sp")
library("foreach")
library("stringr")
library("aws.s3")
#conda install -c conda-forge r-optmatch #r-ggmap r-hrbrthemes r-Hmisc
library("optmatch")
#install.packages("doParallel")
library("doParallel")


In [24]:
#To test, we define the variables manually. For final version, run the commented out section below
#iso3 <-"Bpt"
#iso3 <-"Bce"
#iso3 <-"Bca"
#iso3 <- "Bma"
#iso3 <- "BaE"
#iso3 <- "BaW"
##
#iso3 <- "GHA"
iso3 <- "UGA"
gediwk <- 24
mproc <- 1

In [25]:
#-------------------------------------------------------------------------------
#args = commandArgs(trailingOnly=TRUE)
#if (length(args)==0) {
#  stop("At least one argument must be supplied (input file).n", call.=FALSE)
#} else if (length(args)>=1) {
#  
#  iso3 <- args[1]  #country to process
#  gediwk <- args[2]   #the # of weeks GEDI data to use
#  mproc <- as.integer(args[3])  #the number of cores to use for matching 
#}
#-------------------------------------------------------------------------------

In [26]:
cat("Step 0: Loading global variables for", iso3,"with wk", gediwk, "data \n")

#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"

matching_tifs <- c("wwf_biomes","wwf_ecoreg","lc2000","d2roads", "dcities","dem",
                   "pop_cnt_2000","pop_den_2000","slope", "tt2cities_2000", "wc_prec_1990-1999",
                   "wc_tmax_1990-1999","wc_tavg_1990-1999","wc_tmin_1990-1999" )

ecoreg_key <- read.csv(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))
#unlink(s3_get(paste(f.path,"wwf_ecoregions_key.csv",sep="")))

allPAs <- readRDS(s3_get(paste(f.path,"WDPA_shapefiles/WDPA_polygons/",iso3,"_PA_poly.rds",sep="")))

MCD12Q1 <- rast(s3_get(paste(f.path,"GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif",sep="")))
crs(MCD12Q1)  <- "epsg:6933"

world_region <- rast(s3_get(paste(f.path,"GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif",sep="")))
crs(world_region)  <- "epsg:6933"

s3_get_files(c(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".shx",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".prj",sep=""),
              paste(f.path,"WDPA_countries/shp/",iso3,".dbf",sep="")),confirm = FALSE)
adm <- st_read(s3_get(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep="")))

#s3_path <- paste("/vsis3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/",iso3,".shp",sep="") #Redo this for the gpkg
#adm <- st_read(s3_path)
adm_prj <- project(vect(adm), "epsg:6933")

load(s3_get(paste(f.path,"rf_noclimate.RData",sep="")))
#source(s3_get(paste(f.path,"matching_func.R",sep="")))
source(s3_get(paste(f.path,"matching_func_2024.R",sep="")))


Step 0: Loading global variables for UGA with wk 24 data 


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/wwf_ecoregions_key.csv

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif

ℹ 4 files totaling 18.75 kB will be downloaded to /projects/.local/share/R/s3

→ Downloading 4 files.

→ Got 0 files, downloading 4

→ Got 1 file, downloading 3

→ Got 2 files, downloading 2

Reading layer `UGA' from data source 
  `/projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/WDPA_countries/shp/UGA.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 1 feature and 94 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: 29.54846 ymin: -1.475206 xmax: 35.00647 ymax: 4.219692
Geodetic CRS:  WGS 84


ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/rf_noclimate.RData

ℹ s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/matching_func_2024.R already exists at /projects/.local/share/R/s3/maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/matching_func_2024.R



In [27]:
#STEP4. Set up spatial points data frames (control + each PA) for point matching
# if (file.exists(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",iso3,"_matching_output_wk",gediwk,".RDS", sep=""))){
d_control_local <- readRDS(s3_get(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_prepped_control_wk",gediwk,".RDS",sep="")))
d_control_local <- d_control_local[complete.cases(d_control_local), ]  #filter away non-complete cases w/ NA in control set


In [28]:
print(d_control_local[1,])

                                         wwfbiom                      wwfecoreg
1 Tropical & Subtropical Moist Broadleaf Forests Albertine Rift montane forests
  land_cover d2road d2city elevation popcnt popden slope  tt2city    prec
1   l7_water      0      0       912      0      0     0 114.2573 3031.29
  max_temp mean_temp min_temp      lon        lat status UID
1 29.10585  23.24886 17.71366 29.67324 -0.1529903  FALSE   1


In [29]:
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
#testPAs_fileindex <- read.csv(s3_get(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs_fileindex.csv",sep="")))
#d_PAs <- testPAs_fileindex[!is.na(testPAs_fileindex[,"filename"]),]$filename
#d_PAs[1]
#length(d_PAs)

In [30]:
#d_PAs <- list.files("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_matching_points/GHA/GHA_testPAs/")
d_PAs <- list.files(paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_matching_points/",iso3,"/",iso3,"_testPAs/", sep=""))
d_PAs[1]
length(d_PAs)

[1] "prepped_pa_1435_wk24.RDS"

[1] 426

In [31]:
f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
#if(!dir.exists(paste(f.path0,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep=""))){
  # cat("Matching result dir does not EXISTS\n")
dir.create(file.path(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep="")))

In [32]:
#f.path <- "/projects/my-public-bucket/GEDI_global_PA_v2/"
#f.path <- "s3://maap-ops-workspace/shared/leitoldv/GEDI_global_PA_v2/"
#dir.create(file.path(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",sep="")))

In [33]:
d_PAs

[1] "prepped_pa_1435_wk24.RDS"      "prepped_pa_1436_wk24.RDS"     
  [3] "prepped_pa_1437_wk24.RDS"      "prepped_pa_1438_wk24.RDS"     
  [5] "prepped_pa_1439_wk24.RDS"      "prepped_pa_1440_wk24.RDS"     
  [7] "prepped_pa_1441_wk24.RDS"      "prepped_pa_1443_wk24.RDS"     
  [9] "prepped_pa_1444_wk24.RDS"      "prepped_pa_1445_wk24.RDS"     
 [11] "prepped_pa_1446_wk24.RDS"      "prepped_pa_18437_wk24.RDS"    
 [13] "prepped_pa_18438_wk24.RDS"     "prepped_pa_20556_wk24.RDS"    
 [15] "prepped_pa_28175_wk24.RDS"     "prepped_pa_301308_wk24.RDS"   
 [17] "prepped_pa_3051_wk24.RDS"      "prepped_pa_31275_wk24.RDS"    
 [19] "prepped_pa_313096_wk24.RDS"    "prepped_pa_313105_wk24.RDS"   
 [21] "prepped_pa_313107_wk24.RDS"    "prepped_pa_313108_wk24.RDS"   
 [23] "prepped_pa_313109_wk24.RDS"    "prepped_pa_313110_wk24.RDS"   
 [25] "prepped_pa_313112_wk24.RDS"    "prepped_pa_315097_wk24.RDS"   
 [27] "prepped_pa_315104_wk24.RDS"    "prepped_pa_315107_wk24.RDS"   
 [29] "prepped_pa_315109_wk24.RDS"    "prepped_pa_315111_wk24.RDS"   
 [31] "prepped_pa_315112_wk24.RDS"    "prepped_pa_315116_wk24.RDS"   
 [33] "prepped_pa_315117_wk24.RDS"    "prepped_pa_315118_wk24.RDS"   
 [35] "prepped_pa_315120_wk24.RDS"    "prepped_pa_315124_wk24.RDS"   
 [37] "prepped_pa_315125_wk24.RDS"    "prepped_pa_315136_wk24.RDS"   
 [39] "prepped_pa_315141_wk24.RDS"    "prepped_pa_315143_wk24.RDS"   
 [41] "prepped_pa_315159_wk24.RDS"    "prepped_pa_315161_wk24.RDS"   
 [43] "prepped_pa_315172_wk24.RDS"    "prepped_pa_315178_wk24.RDS"   
 [45] "prepped_pa_315195_wk24.RDS"    "prepped_pa_315209_wk24.RDS"   
 [47] "prepped_pa_315210_wk24.RDS"    "prepped_pa_315216_wk24.RDS"   
 [49] "prepped_pa_315231_wk24.RDS"    "prepped_pa_315235_wk24.RDS"   
 [51] "prepped_pa_315243_wk24.RDS"    "prepped_pa_315245_wk24.RDS"   
 [53] "prepped_pa_315247_wk24.RDS"    "prepped_pa_315252_wk24.RDS"   
 [55] "prepped_pa_315285_wk24.RDS"    "prepped_pa_315291_wk24.RDS"   
 [57] "prepped_pa_315295_wk24.RDS"    "prepped_pa_315299_wk24.RDS"   
 [59] "prepped_pa_315300_wk24.RDS"    "prepped_pa_315346_wk24.RDS"   
 [61] "prepped_pa_315380_wk24.RDS"    "prepped_pa_315384_wk24.RDS"   
 [63] "prepped_pa_315391_wk24.RDS"    "prepped_pa_315405_wk24.RDS"   
 [65] "prepped_pa_315408_wk24.RDS"    "prepped_pa_315426_wk24.RDS"   
 [67] "prepped_pa_315444_wk24.RDS"    "prepped_pa_315462_wk24.RDS"   
 [69] "prepped_pa_317282_wk24.RDS"    "prepped_pa_33147_wk24.RDS"    
 [71] "prepped_pa_34804_wk24.RDS"     "prepped_pa_34805_wk24.RDS"    
 [73] "prepped_pa_34808_wk24.RDS"     "prepped_pa_354017_wk24.RDS"   
 [75] "prepped_pa_39980_wk24.RDS"     "prepped_pa_39981_wk24.RDS"    
 [77] "prepped_pa_39985_wk24.RDS"     "prepped_pa_39987_wk24.RDS"    
 [79] "prepped_pa_39988_wk24.RDS"     "prepped_pa_39989_wk24.RDS"    
 [81] "prepped_pa_39990_wk24.RDS"     "prepped_pa_39991_wk24.RDS"    
 [83] "prepped_pa_39992_wk24.RDS"     "prepped_pa_39993_wk24.RDS"    
 [85] "prepped_pa_39994_wk24.RDS"     "prepped_pa_39997_wk24.RDS"    
 [87] "prepped_pa_39998_wk24.RDS"     "prepped_pa_39999_wk24.RDS"    
 [89] "prepped_pa_40000_wk24.RDS"     "prepped_pa_40001_wk24.RDS"    
 [91] "prepped_pa_40002_wk24.RDS"     "prepped_pa_40006_wk24.RDS"    
 [93] "prepped_pa_40007_wk24.RDS"     "prepped_pa_40008_wk24.RDS"    
 [95] "prepped_pa_40009_wk24.RDS"     "prepped_pa_40012_wk24.RDS"    
 [97] "prepped_pa_40014_wk24.RDS"     "prepped_pa_40015_wk24.RDS"    
 [99] "prepped_pa_40017_wk24.RDS"     "prepped_pa_40019_wk24.RDS"    
[101] "prepped_pa_40020_wk24.RDS"     "prepped_pa_40023_wk24.RDS"    
[103] "prepped_pa_40024_wk24.RDS"     "prepped_pa_40025_wk24.RDS"    
[105] "prepped_pa_40027_wk24.RDS"     "prepped_pa_40034_wk24.RDS"    
[107] "prepped_pa_40035_wk24.RDS"     "prepped_pa_40036_wk24.RDS"    
[109] "prepped_pa_40042_wk24.RDS"     "prepped_pa_40044_wk24.RDS"    
[111] "prepped_pa_40045_wk24.RDS"     "prepped_pa_40046_wk24.RDS"    
[113] "prepped_pa_40048_wk24.RDS"     "prepped_pa_40052_wk24.RDS"    
[115] "prepp

In [34]:
#Initialize an empty list to store filenames
#output_filenames <- cbind(d_PAs, rep(NA, length(d_PAs)))
#colnames(output_filenames) <- c("prepped_PA", "matched_PA")
#matched_PA <- c()

registerDoParallel(mproc)
# cat("Parallel processing",getDoParWorkers(),"PAs \n")
startTime <- Sys.time()
#d_PAs <- list.files(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/", sep=""), pattern=paste("wk",gediwk,sep=""), full.names=FALSE)

foreach(this_pa=d_PAs,.combine = foreach_rbind, .packages=c('sp','magrittr', 'dplyr','tidyr','optmatch','doParallel')) %dopar% {
  pa <- this_pa
  #id_pa <- pa %>%str_split("_") %>% unlist %>% .[[3]]
  id_pa <- this_pa%>%readr::parse_number() %>% unique() #%>%str_split("_") %>% unlist %>% .[4] #With new files, check where PA ID is in string
  cat(id_pa, "in",iso3,"\n")
  cat("No.", match(pa,d_PAs),"of total",length(d_PAs),"PAs in ", iso3, "\n" )
  d_pa <- readRDS(paste(f.path,"WDPA_matching_points/",iso3,"/",iso3,"_testPAs/",pa, sep=""))
  cat(iso3, "pa no.",id_pa, "has",nrow(d_pa)," of treatment \n")
  d_filtered_prop <- tryCatch(propensity_filter(d_pa, d_control_local), error=function(e) return(NA))  #return a df of control and treatment after complete cases and propensity filters are applied
  cat("Propensity score filtered DF dimension is",dim(d_filtered_prop),"\n")
  d_wocat_all <- tryCatch(filter(d_filtered_prop, status),error=function(e) return(NA))
  d_control_all <- tryCatch(filter(d_filtered_prop, !status),error=function(e) return(NA))
  
  n_control <- dim(d_control_all)[1]
  # ids_all <- d_control_all$UID   #seq(1,n_control)
  ids_all0 <- tryCatch(d_control_all$UID, error=function(e) return(NA))
  ids_all <- d_control_all$UID
  set.seed(125)
  cat("Using number of cores:",getDoParWorkers(),"\n")
  N <- ceiling(nrow(d_wocat_all)/300)
  l <- tryCatch(split(d_wocat_all, sample(1:N, nrow(d_wocat_all), replace=TRUE)),error=function(e) return(NULL))
  # l <- tryCatch(split(d_wocat_all, (as.numeric(rownames(d_wocat_all))-1) %/% 300),error=function(e) return(0))
  
#  if (length(l)<50 && length(l)>0 ){
  if (length(l)<900 && length(l)>0 ){
    pa_match <- data.frame()
    for (pa_c in 1:length(l)){
      ids_all <- d_control_all$UID
      cat("chunk",pa_c,"of PA", id_pa,"\n")
      
      d_wocat_chunk <- l[[pa_c]]
      # #sample the control dataset to the size of the sample dataset, keep unsampled ids to iterate until full number of matches found
      n_treatment <- dim(d_wocat_chunk)[1]
      
      t <- ifelse(floor(n_control/n_treatment)<=7, ifelse(floor(n_control/n_treatment)<1, 1,floor(n_control/n_treatment)),7)   #floor(n_control/n_treatment))
      n_sample <- round(n_treatment*t)    #now the n_control is 1.4 times the number of n_treatment, 7 will set the if ststament below to flase
      m_all2_out <- data.frame()
      Bscore <- data.frame()
      n_matches <- 0
      tryCatch(
        while(n_matches < n_treatment){
          n_ids <- length(ids_all)
          cat("n ids",n_ids,"\n")
          if(n_ids > n_sample){
            set.seed(125)
            sample_ids_bar <- sample(ids_all, n_sample)
            sample_ids <- sample(ids_all0, n_sample)
            d_control_sample <- d_control_all[d_control_all$UID %in% sample_ids,]
            ids_all <-setdiff(ids_all, sample_ids_bar)    #ids_all[-sample_ids]
            cat("protected uid", head(d_wocat_chunk$UID),"\n")
            # All approaches
            new_d <- tryCatch(rbind(d_wocat_chunk,d_control_sample),error=function(e) return(NULL))
            # new_d <- tryCatch(rbind(d_wocat_chunk,d_control_all),error=function(e) return(NULL))
            #create a smaller distance matrix
            m_all <- tryCatch(match_wocat(new_d, pid=id_pa),error=function(e) return(NULL))
            # m_all <- match_wocat(new_d)
            m_all2 <- tryCatch(m_all[1,],error=function(e) return(NULL))
            # m_all2 <- m_all[1,]
            n_matches_temp <- tryCatch(nrow(m_all2$df),error=function(e) return(NULL))
            # n_matches_temp <- nrow(m_all2$df)
            if(!is.null(n_matches_temp)){
              # n_matches <- n_matches + nrow(m_all2$df)
              m_all2$df$pa_id <- rep(id_pa,n_matches_temp)
              m_all2_out <- rbind(m_all2_out, m_all2$df)
              matched_protected <- m_all2$df %>% dplyr::filter(status==TRUE)
              matched_control <- m_all2$df %>% dplyr::filter(status==FALSE)
              cat("matched_protected", nrow(matched_protected),"\n")
              n_matches <- n_matches + nrow(matched_protected)
              d_wocat_chunk <- d_wocat_chunk[-(match(matched_protected$UID,d_wocat_chunk$UID)),]
              # d_control_all <- d_control_all[-(match(matched_control$UID,d_control$UID)),]
            } 
            # ids_all <-setdiff(ids_all, sample_ids)
            ids_all0 <-setdiff(ids_all0, matched_control$UID)
            # else {
            #   n_treatment <- 0  #if not macthes are found in this sampling
            # }
          } else {n_treatment <- n_matches}
        }, error=function(e) return(NULL))
      # ids_all0 <-setdiff(ids_all0, matched_control$UID)
      match_score <- m_all2_out
      cat(table(match_score$status),"\n")
      pa_match <- rbind(pa_match,match_score)
    }
#  } else if (length(l)>=50){
  } else if (length(l)>=900){
    registerDoParallel(3)
    #pa_match <- foreach(pa_c=length(l), .combine = foreach_rbind, .packages=c('sp','magrittr', 'dplyr','tidyr','optmatch','doParallel'))%dopar%{
    pa_match <- foreach(pa_c=1:length(l), .combine = foreach_rbind, .packages=c('sp','magrittr', 'dplyr','tidyr','optmatch','doParallel'))%dopar%{
      cat("Matching treatment chunk", pa_c, "out of", length(l), "for PA", id_pa,"\n")
      cat("chunk",pa_c,"out of ",length(l), "chunks of PA", id_pa,"\n")
      cat("head control",head(ids_all0),"\n")
      d_wocat_chunk <- l[[pa_c]]
      # #sample the control dataset to the size of the sample dataset, keep unsampled ids to iterate until full number of matches found
      n_treatment <- dim(d_wocat_chunk)[1]
      cat( "n control", length(ids_all0),"\n")
      t <- ifelse(floor(n_control/n_treatment)<=7, ifelse(floor(n_control/n_treatment)<1, 1,floor(n_control/n_treatment)),7)   #floor(n_control/n_treatment))
      n_sample <- round(n_treatment*t)    #now the n_control is 1.4 times the number of n_treatment, 7 will set the if ststament below to flase
      m_all2_out <- data.frame()
      Bscore <- data.frame()
      n_matches <- 0
      
      tryCatch(
        while(n_matches < n_treatment){
          
          n_ids <- length(ids_all0)
          cat("n ids",n_ids,"\n")
          if(n_ids > n_sample){
            set.seed(125)
            sample_ids_bar <- sample(ids_all, n_sample)
            sample_ids <- sample(ids_all0, n_sample)
            d_control_sample <- d_control_all[d_control_all$UID %in% sample_ids,]
            ids_all <-setdiff(ids_all, sample_ids)    #ids_all[-sample_ids]
            cat("protected uid", head(d_wocat_chunk$UID),"\n")
            # All approaches
            new_d <- tryCatch(rbind(d_wocat_chunk,d_control_sample),error=function(e) return(NULL))
            # new_d <- tryCatch(rbind(d_wocat_chunk,d_control_all),error=function(e) return(NULL))
            
            #create a smaller distance matrix
            m_all <- tryCatch(match_wocat(new_d, pid=id_pa),error=function(e) return(NULL))
            # m_all <- match_wocat(new_d)
            m_all2 <- tryCatch(m_all[1,],error=function(e) return(NULL))
            # m_all2 <- m_all[1,]
            n_matches_temp <- tryCatch(nrow(m_all2$df),error=function(e) return(NULL))
            # n_matches_temp <- nrow(m_all2$df)
            if(!is.null(n_matches_temp)){
              # n_matches <- n_matches + nrow(m_all2$df)
              m_all2$df$pa_id <- rep(id_pa,n_matches_temp)
              m_all2_out <- rbind(m_all2_out, m_all2$df)
              matched_protected <- m_all2$df %>% dplyr::filter(status==TRUE)
              matched_control <- m_all2$df %>% dplyr::filter(status==FALSE)
              cat("matched_protected", nrow(matched_protected),"\n")
              n_matches <- n_matches + nrow(matched_protected)
              d_wocat_chunk <- d_wocat_chunk[-(match(matched_protected$UID,d_wocat_chunk$UID)),]
              # d_control_all <- d_control_all[-(match(matched_control$UID,d_control$UID)),]
              # 
            } 
            ids_all0 <-setdiff(ids_all0, matched_control$UID)
            cat( "n control", length(ids_all0),"\n")
            
            # else {
            #   n_treatment <- 0  #if not macthes are found in this sampling
            # }
          } else {n_treatment <- n_matches}
        }, error=function(e) return(NULL))
      # ids_all0 <-setdiff(ids_all0, matched_control$UID)
      match_score <- m_all2_out
      cat(table(match_score$status),"\n")
      return(match_score)
    }
    stopImplicitCluster()
  } else{
    pa_match <- NULL
  }

    output_filename <- paste(iso3,"_pa_", id_pa,"_matching_results_wk",gediwk,".RDS", sep="")
    saveRDS(pa_match, file=paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"/",output_filename, sep=""))
  
  # Append the filename to the list
#  matched_PA <- c(matched_PA, output_filename)
#  print(output_filename)
                                       
  cat("Results exported for PA", id_pa, "\n")
  rm(pa_match)
  return(NULL)
}

tElapsed <- Sys.time()-startTime
# cat(tElapsed, "for matching all PAs in", iso3,"\n")
stopImplicitCluster()
cat("Done matching for",iso3,". Finishing...\n")

# Write the list of filenames to a CSV file
#output_filenames$matched_PA <- matched_PA
#write.csv(output_filenames, file=paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,"_matching_result_filenames.csv", sep=""), row.names = FALSE, col.names = FALSE)


1435 in UGA 
No. 1 of total 426 PAs in  UGA 
UGA pa no. 1435 has 2148  of treatment 
Propensity score filtered DF dimension is 52247 19 
Using number of cores: 1 
chunk 1 of PA 1435 
n ids 50105 
protected uid 7 8 23 25 49 51 
matched_protected 77 
n ids 48208 
protected uid 7 8 23 25 51 56 
matched_protected 150 
n ids 46311 
protected uid 129 153 187 194 208 209 
n ids 44414 
protected uid 129 153 187 194 208 209 
n ids 42517 
protected uid 129 153 187 194 208 209 
n ids 40620 
protected uid 129 153 187 194 208 209 
n ids 38723 
protected uid 129 153 187 194 208 209 
n ids 36826 
protected uid 129 153 187 194 208 209 
n ids 34929 
protected uid 129 153 187 194 208 209 
n ids 33032 
protected uid 129 153 187 194 208 209 
n ids 31135 
protected uid 129 153 187 194 208 209 
n ids 29238 
protected uid 129 153 187 194 208 209 
n ids 27341 
protected uid 129 153 187 194 208 209 
n ids 25444 
protected uid 129 153 187 194 208 209 
n ids 23547 
protected uid 129 153 187 194 208 209 
n ids 21

NULL

Done matching for UGA . Finishing...


In [35]:
f.path

[1] "/projects/my-public-bucket/GEDI_global_PA_v2/"

In [36]:
output_path <- paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_matching_results/",iso3,"_wk24",sep="")

In [37]:
list.files(output_path, full.names=F)

[1] "UGA_pa_1435_matching_results_wk24.RDS"     
  [2] "UGA_pa_1436_matching_results_wk24.RDS"     
  [3] "UGA_pa_1437_matching_results_wk24.RDS"     
  [4] "UGA_pa_1438_matching_results_wk24.RDS"     
  [5] "UGA_pa_1439_matching_results_wk24.RDS"     
  [6] "UGA_pa_1440_matching_results_wk24.RDS"     
  [7] "UGA_pa_1441_matching_results_wk24.RDS"     
  [8] "UGA_pa_1443_matching_results_wk24.RDS"     
  [9] "UGA_pa_1444_matching_results_wk24.RDS"     
 [10] "UGA_pa_1445_matching_results_wk24.RDS"     
 [11] "UGA_pa_1446_matching_results_wk24.RDS"     
 [12] "UGA_pa_18437_matching_results_wk24.RDS"    
 [13] "UGA_pa_18438_matching_results_wk24.RDS"    
 [14] "UGA_pa_20556_matching_results_wk24.RDS"    
 [15] "UGA_pa_28175_matching_results_wk24.RDS"    
 [16] "UGA_pa_301308_matching_results_wk24.RDS"   
 [17] "UGA_pa_3051_matching_results_wk24.RDS"     
 [18] "UGA_pa_31275_matching_results_wk24.RDS"    
 [19] "UGA_pa_313096_matching_results_wk24.RDS"   
 [20] "UGA_pa_313105_matching_results_wk24.RDS"   
 [21] "UGA_pa_313107_matching_results_wk24.RDS"   
 [22] "UGA_pa_313108_matching_results_wk24.RDS"   
 [23] "UGA_pa_313109_matching_results_wk24.RDS"   
 [24] "UGA_pa_313110_matching_results_wk24.RDS"   
 [25] "UGA_pa_313112_matching_results_wk24.RDS"   
 [26] "UGA_pa_315097_matching_results_wk24.RDS"   
 [27] "UGA_pa_315104_matching_results_wk24.RDS"   
 [28] "UGA_pa_315107_matching_results_wk24.RDS"   
 [29] "UGA_pa_315109_matching_results_wk24.RDS"   
 [30] "UGA_pa_315111_matching_results_wk24.RDS"   
 [31] "UGA_pa_315112_matching_results_wk24.RDS"   
 [32] "UGA_pa_315116_matching_results_wk24.RDS"   
 [33] "UGA_pa_315117_matching_results_wk24.RDS"   
 [34] "UGA_pa_315118_matching_results_wk24.RDS"   
 [35] "UGA_pa_315120_matching_results_wk24.RDS"   
 [36] "UGA_pa_315124_matching_results_wk24.RDS"   
 [37] "UGA_pa_315125_matching_results_wk24.RDS"   
 [38] "UGA_pa_315136_matching_results_wk24.RDS"   
 [39] "UGA_pa_315141_matching_results_wk24.RDS"   
 [40] "UGA_pa_315143_matching_results_wk24.RDS"   
 [41] "UGA_pa_315159_matching_results_wk24.RDS"   
 [42] "UGA_pa_315161_matching_results_wk24.RDS"   
 [43] "UGA_pa_315172_matching_results_wk24.RDS"   
 [44] "UGA_pa_315178_matching_results_wk24.RDS"   
 [45] "UGA_pa_315195_matching_results_wk24.RDS"   
 [46] "UGA_pa_315209_matching_results_wk24.RDS"   
 [47] "UGA_pa_315210_matching_results_wk24.RDS"   
 [48] "UGA_pa_315216_matching_results_wk24.RDS"   
 [49] "UGA_pa_315231_matching_results_wk24.RDS"   
 [50] "UGA_pa_315235_matching_results_wk24.RDS"   
 [51] "UGA_pa_315243_matching_results_wk24.RDS"   
 [52] "UGA_pa_315245_matching_results_wk24.RDS"   
 [53] "UGA_pa_315247_matching_results_wk24.RDS"   
 [54] "UGA_pa_315252_matching_results_wk24.RDS"   
 [55] "UGA_pa_315285_matching_results_wk24.RDS"   
 [56] "UGA_pa_315291_matching_results_wk24.RDS"   
 [57] "UGA_pa_315295_matching_results_wk24.RDS"   
 [58] "UGA_pa_315299_matching_results_wk24.RDS"   
 [59] "UGA_pa_315300_matching_results_wk24.RDS"   
 [60] "UGA_pa_315346_matching_results_wk24.RDS"   
 [61] "UGA_pa_315380_matching_results_wk24.RDS"   
 [62] "UGA_pa_315384_matching_results_wk24.RDS"   
 [63] "UGA_pa_315391_matching_results_wk24.RDS"   
 [64] "UGA_pa_315405_matching_results_wk24.RDS"   
 [65] "UGA_pa_315408_matching_results_wk24.RDS"   
 [66] "UGA_pa_315426_matching_results_wk24.RDS"   
 [67] "UGA_pa_315444_matching_results_wk24.RDS"   
 [68] "UGA_pa_315462_matching_results_wk24.RDS"   
 [69] "UGA_pa_317282_matching_results_wk24.RDS"   
 [70] "UGA_pa_33147_matching_results_wk24.RDS"    
 [71] "UGA_pa_34804_matching_results_wk24.RDS"    
 [72] "UGA_pa_34805_matching_results_wk24.RDS"    
 [73] "UGA_pa_34808_matching_results_wk24.RDS"    
 [74] "UGA_pa_354017_matching_results_wk24.RDS"   
 [75] "UGA_pa_39980_matching_results_wk24.RDS"    
 [76] "UGA_pa_39981_matching_results_wk24.RDS"    
 [77] "UGA_pa_39985_matching_results_wk24.RDS"    
 [78] "UGA_pa_39987_matching_results_wk24.RDS"    
 [79] "UGA_pa_39988_matc

In [38]:
write.csv(data.frame(list.files(output_path, full.names=F)),
          file=paste("/projects/my-public-bucket/GEDI_global_PA_v2/WDPA_matching_results/",iso3,"_wk24_filelist.csv",sep=""))

In [39]:
##
matching_results <- list.files(output_path, full.names=T)
length(matching_results)
##
for(i in 1:length(matching_results)){
    m_data <- readRDS(matching_results[i])
    print(nrow(m_data))
    }


[1] 426

[1] 3440
[1] 3608
[1] 1558
[1] 614
[1] 1348
[1] 1078
[1] 562
[1] 50
[1] 68
[1] 312
[1] 30
[1] 116
[1] 0
[1] 124
[1] 74
[1] 0
[1] 1210
[1] 214
[1] 3814
[1] 2090
[1] 1522
[1] 4
[1] 0
[1] 4
[1] 364
[1] 172
[1] 0
[1] 4
[1] 266
[1] 20
[1] 4
[1] 0
[1] 0
[1] 0
[1] 0
[1] 8
[1] 0
[1] 6
[1] 20
[1] 324
[1] 10
[1] 22
[1] 4
[1] 170
[1] 2
[1] 50
[1] 0
[1] 116
[1] 4
[1] 88
[1] 0
[1] 8
[1] 62
[1] 4
[1] 14
NULL
[1] 0
[1] 2
[1] 4
[1] 14
[1] 8
[1] 8
[1] 0
[1] 4
[1] 4
[1] 4
[1] 44
[1] 102
[1] 4
[1] 6
[1] 1522
[1] 64
[1] 140
[1] 0
[1] 102
[1] 28
[1] 426
[1] 12
[1] 0
[1] 14
[1] 20
[1] 4
[1] 12
[1] 62
[1] 4
[1] 0
[1] 0
[1] 4
[1] 2
[1] 18
[1] 466
[1] 72
[1] 6
[1] 12
[1] 66
[1] 0
[1] 0
[1] 8
[1] 192
[1] 10
[1] 6
[1] 10
[1] 82
[1] 2
[1] 276
[1] 16
[1] 182
[1] 84
[1] 0
[1] 0
[1] 0
[1] 0
[1] 20
[1] 174
[1] 2
[1] 10
[1] 12
[1] 6
[1] 2
[1] 6
[1] 14
[1] 0
[1] 4
[1] 6
[1] 0
[1] 2
[1] 96
[1] 0
[1] 6
[1] 4
[1] 10
[1] 14
[1] 0
[1] 6
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
[1] 0
NULL
[1] 0
[1] 0
[1] 0
[1] 16
[1] 306
[1] 134
